# 緯度経度のデータ獲得を試みる！

In [184]:
import pandas as pd
import category_encoders as ce
from cmath import nan
pd.set_option('display.max_columns', 100)

In [185]:
#所在地のテキスト整形用関数
def func_place(block):
    target2 = '丁'
    target3 = '町'
    try:
        place = block
        if '八丁堀' in place:
            return '八丁堀'

        elif '町屋' in place:
            return '町屋'

        elif '関町南' in place:
            return '関町南'

        elif '関町北' in place:
            return '関町北'

        elif '関町東' in place:
            return '関町東'

        elif '関町西' in place:
            return '関町西'

        elif target3 in block:
            place = block.split(target3)[0]
            place = place + target3

        elif target2 in block:
            place = block.split(target2)[0]
            place = place[:-1]#○丁目の○の部分を除く
        return place
        
    except:
        return block

#「大字・丁目名」「緯度」「経度」の内容を成形し、データフレーム型にして返す関数
def get_ward_infos(df_ward):
    ward_infos = []
    for i in range(len(df_ward)):
        
        block = df_ward['大字・丁目名']
        address = func_place(block.loc[i])
        address = df_ward['市区町村名'].loc[0] + address
        latitude = df_ward['緯度'].loc[i]
        longitude = df_ward['経度'].loc[i]
        ward_info = [address, latitude, longitude]
        ward_infos.append(ward_info)
    
    ward_infos = pd.DataFrame(ward_infos)
    ward_infos = ward_infos.rename(columns={0:'区町名',1:'緯度',2:'経度'})#列名の振り直し
    return ward_infos

#受け取った"△△町○○丁目"の○○丁目ごとの平均緯度と経度を計算し、データフレーム型にして返す関数
def get_mean_latitude_longitude(ward_infos):

    ##「区町名」でグループ化を施した上で、一旦ふつうのリストに区町名を順番を保って保存する。
    ###その上でグループ化結果の区町名ごとの平均化された緯度経度のデータと結合させる。
    ####まどろっこしいが、これより簡単な方法がわからない・・・
    grouped = ward_infos.groupby('区町名')#ここでグループ化
    blocks = list(grouped.groups)#順番を保った上で「区町名」を全て取り出す
    mean_latitude_longitudes = []
    i = 0
    for block in blocks:
        mean_latitude = grouped.mean()['緯度'][i]#区町名の順番ずつに、その平均緯度を取得する。
        mean_longitude = grouped.mean()['経度'][i]#区町名の順番ずつに、その平均緯度を取得する。
        mean_latitude_longitudes.append([block, mean_latitude, mean_longitude])
        i += 1

    mean_latitude_longitudes = pd.DataFrame(mean_latitude_longitudes)#ここで改めてデータフレーム型に変換する。
    mean_latitude_longitudes.columns = ['区町名','平均緯度','平均経度']

    return mean_latitude_longitudes

### 千代田区

In [186]:
df_chiyoda = pd.read_csv('../data/outer_data/13101_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_chiyoda)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/chiyoda_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

            区町名       平均緯度        平均経度
0       千代田区一ツ橋  35.693013  139.757626
1       千代田区一番町  35.687195  139.742098
2       千代田区三番町  35.690541  139.743154
3       千代田区丸の内  35.680021  139.764217
4       千代田区九段北  35.695620  139.747396
5       千代田区九段南  35.692930  139.745005
6       千代田区二番町  35.686099  139.736311
7       千代田区五番町  35.689763  139.734653
8       千代田区六番町  35.687294  139.733599
9       千代田区内幸町  35.670189  139.756925
10      千代田区内神田  35.691074  139.767893
11    千代田区北の丸公園  35.691791  139.752152
12      千代田区千代田  35.685460  139.756032
13      千代田区四番町  35.688964  139.737905
14      千代田区外神田  35.701603  139.770499
15      千代田区大手町  35.686804  139.765915
16      千代田区富士見  35.698456  139.745085
17      千代田区岩本町  35.693873  139.777159
18      千代田区平河町  35.681287  139.740528
19    千代田区日比谷公園  35.673587  139.756055
20      千代田区有楽町  35.674029  139.761476
21      千代田区東神田  35.695642  139.780698
22      千代田区永田町  35.675433  139.742837
23     千代田区皇居外苑  35.679988  139.757859
24    千代田区神田三崎町  35.70077

## 中央区

In [187]:
df_chuo = pd.read_csv('../data/outer_data/13102_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_chuo)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/chuo_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

           区町名       平均緯度        平均経度
0        中央区京橋  35.676841  139.771413
1         中央区佃  35.666494  139.785102
2        中央区入船  35.671822  139.777005
3       中央区八丁堀  35.675819  139.776431
4       中央区八重洲  35.680272  139.769743
5       中央区勝どき  35.658015  139.775604
6        中央区新富  35.672264  139.774246
7        中央区新川  35.675837  139.782691
8       中央区日本橋  35.681475  139.774430
9     中央区日本橋中洲  35.683475  139.790632
10   中央区日本橋久松町  35.689328  139.784409
11   中央区日本橋人形町  35.685701  139.783562
12    中央区日本橋兜町  35.680821  139.777491
13   中央区日本橋堀留町  35.688184  139.780390
14  中央区日本橋大伝馬町  35.690302  139.779851
15    中央区日本橋室町  35.686910  139.773942
16   中央区日本橋富沢町  35.688834  139.782949
17  中央区日本橋小伝馬町  35.691517  139.778864
18   中央区日本橋小網町  35.682752  139.781023
19   中央区日本橋小舟町  35.686831  139.778313
20    中央区日本橋本町  35.688210  139.775890
21   中央区日本橋本石町  35.687667  139.770770
22   中央区日本橋横山町  35.692800  139.783297
23    中央区日本橋浜町  35.686969  139.788250
24   中央区日本橋箱崎町  35.680189  139.786256
25   中央区日本橋茅

## 港区

In [188]:
df_minato = pd.read_csv('../data/outer_data/13103_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_minato)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/minato_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0      港区三田  35.646916  139.740326
1     港区元赤坂  35.678121  139.729303
2     港区元麻布  35.654943  139.730114
3     港区六本木  35.663042  139.733496
4     港区北青山  35.669208  139.715365
5     港区南青山  35.665712  139.719346
6     港区南麻布  35.650036  139.730075
7      港区台場  35.630170  139.775508
8      港区愛宕  35.664185  139.748714
9      港区新橋  35.664538  139.755752
10    港区東新橋  35.662165  139.758175
11    港区東麻布  35.656683  139.741593
12    港区浜松町  35.656827  139.756105
13     港区海岸  35.645896  139.757744
14     港区港南  35.630003  139.747657
15     港区白金  35.644208  139.727796
16    港区白金台  35.637126  139.725354
17      港区芝  35.650137  139.749223
18    港区芝公園  35.657337  139.749146
19    港区芝大門  35.656817  139.753690
20     港区芝浦  35.643495  139.750390
21    港区虎ノ門  35.665792  139.746798
22    港区西新橋  35.666401  139.752046
23    港区西麻布  35.659535  139.723083
24     港区赤坂  35.671482  139.735803
25     港区高輪  35.634413  139.735079
26   港区麻布十番  35.654918  139.735297
27    港区麻布台  35.6603

## 新宿区

In [189]:
df_shinjuku = pd.read_csv('../data/outer_data/13104_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_shinjuku)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/shinjuku_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    新宿区上落合  35.713366  139.687651
1   新宿区下宮比町  35.703173  139.743654
2    新宿区下落合  35.719061  139.699511
3     新宿区中井  35.716280  139.684147
4     新宿区中町  35.699172  139.735741
..      ...        ...         ...
89  新宿区赤城元町  35.704771  139.736290
90  新宿区霞ヶ丘町  35.676751  139.715380
91   新宿区須賀町  35.685048  139.722330
92  新宿区馬場下町  35.705963  139.719283
93  新宿区高田馬場  35.712085  139.701709

[94 rows x 3 columns]


## 文京区

In [190]:
df_bunkyo = pd.read_csv('../data/outer_data/13105_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_bunkyo)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/bunkyo_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

       区町名       平均緯度        平均経度
0    文京区千石  35.726566  139.740500
1   文京区千駄木  35.726430  139.760721
2    文京区向丘  35.720711  139.756626
3    文京区大塚  35.722695  139.731254
4   文京区小日向  35.713626  139.735742
5   文京区小石川  35.714022  139.746565
6    文京区弥生  35.717070  139.762539
7    文京区後楽  35.705285  139.746981
8    文京区春日  35.709948  139.745375
9    文京区本郷  35.708088  139.758662
10  文京区本駒込  35.730479  139.752025
11   文京区根津  35.718726  139.764696
12   文京区水道  35.709790  139.738589
13   文京区湯島  35.705772  139.767768
14   文京区白山  35.719624  139.749249
15  文京区目白台  35.716257  139.722683
16   文京区西片  35.714533  139.755114
17   文京区関口  35.710779  139.728701
18   文京区音羽  35.715534  139.729456


## 台東区

In [191]:
df_taito = pd.read_csv('../data/outer_data/13106_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_taito)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/taito_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    台東区三ノ輪  35.728591  139.793102
1     台東区三筋  35.704831  139.786294
2     台東区上野  35.707751  139.774252
3   台東区上野公園  35.715409  139.773548
4   台東区上野桜木  35.721992  139.773132
5     台東区下谷  35.722236  139.784490
6     台東区今戸  35.720876  139.804260
7    台東区元浅草  35.708524  139.785197
8     台東区入谷  35.720738  139.787446
9    台東区北上野  35.717359  139.783779
10    台東区千束  35.721856  139.793146
11    台東区台東  35.703968  139.778770
12     台東区寿  35.707771  139.791320
13    台東区小島  35.705112  139.783156
14   台東区日本堤  35.727027  139.797548
15   台東区東上野  35.711552  139.780891
16   台東区東浅草  35.723247  139.800189
17   台東区松が谷  35.715188  139.787231
18    台東区柳橋  35.697568  139.788036
19    台東区根岸  35.724878  139.781962
20    台東区橋場  35.725546  139.806412
21   台東区池之端  35.716235  139.768111
22    台東区浅草  35.717140  139.796679
23   台東区浅草橋  35.699437  139.784298
24    台東区清川  35.726178  139.802551
25   台東区秋葉原  35.702157  139.774823
26    台東区竜泉  35.725107  139.790883
27   台東区花川戸  35.7140

## 墨田区

In [192]:
df_sumida = pd.read_csv('../data/outer_data/13107_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_sumida)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/sumida_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

       区町名       平均緯度        平均経度
0    墨田区両国  35.693782  139.793844
1    墨田区亀沢  35.697485  139.802781
2    墨田区京島  35.714988  139.821750
3    墨田区八広  35.721864  139.827099
4    墨田区千歳  35.690660  139.795423
5    墨田区向島  35.715307  139.809668
6   墨田区吾妻橋  35.709125  139.802671
7    墨田区堤通  35.728662  139.813095
8    墨田区墨田  35.732323  139.820000
9    墨田区太平  35.701250  139.813067
10   墨田区押上  35.711793  139.816344
11   墨田区文花  35.709997  139.823998
12   墨田区本所  35.703629  139.802003
13  墨田区東向島  35.723507  139.818036
14  墨田区東墨田  35.719791  139.832887
15  墨田区東駒形  35.706534  139.802415
16   墨田区業平  35.707759  139.813142
17   墨田区横川  35.704098  139.813356
18   墨田区横網  35.699221  139.795094
19  墨田区江東橋  35.694189  139.813503
20   墨田区石原  35.700399  139.802292
21   墨田区立川  35.691127  139.803246
22   墨田区立花  35.709107  139.831919
23    墨田区緑  35.694404  139.803346
24   墨田区菊川  35.688757  139.804926
25   墨田区錦糸  35.698333  139.813629


## 江東区

In [193]:
df_koto = pd.read_csv('../data/outer_data/13108_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_koto)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/koto_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0     江東区三好  35.680333  139.804371
1     江東区亀戸  35.699087  139.829639
2     江東区住吉  35.688715  139.814564
3     江東区佐賀  35.677670  139.790959
4     江東区冬木  35.674382  139.801248
5     江東区北砂  35.680751  139.831385
6     江東区千田  35.681293  139.815687
7     江東区千石  35.678304  139.814748
8     江東区南砂  35.673325  139.829516
9    江東区古石場  35.667395  139.798093
10    江東区塩浜  35.662957  139.809437
11   江東区夢の島  35.649816  139.829405
12    江東区大島  35.689878  139.833590
13    江東区富岡  35.671335  139.799272
14    江東区常盤  35.684996  139.796131
15    江東区平野  35.677800  139.804171
16    江東区扇橋  35.683300  139.815557
17   江東区新大橋  35.688360  139.794500
18   江東区新木場  35.640663  139.830971
19    江東区新砂  35.664537  139.829367
20    江東区有明  35.630492  139.791744
21    江東区木場  35.670500  139.805442
22    江東区東砂  35.677075  139.841350
23    江東区東陽  35.670073  139.812773
24    江東区東雲  35.643919  139.801929
25    江東区枝川  35.657256  139.806290
26    江東区森下  35.686811  139.802383
27    江東区毛利  35.6915

## 品川区

In [194]:
df_shinagawa = pd.read_csv('../data/outer_data/13109_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_shinagawa)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/shinagawa_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    品川区上大崎  35.634704  139.718958
1     品川区中延  35.606883  139.710445
2     品川区二葉  35.604245  139.722216
3     品川区八潮  35.599595  139.755807
4     品川区勝島  35.596454  139.743155
5    品川区北品川  35.620826  139.738821
6    品川区南品川  35.611711  139.740684
7    品川区南大井  35.593033  139.734899
8     品川区大井  35.600886  139.730045
9     品川区大崎  35.619492  139.725456
10    品川区小山  35.617213  139.702872
11   品川区小山台  35.623759  139.703698
12    品川区平塚  35.615285  139.714011
13    品川区広町  35.613461  139.733732
14    品川区戸越  35.611923  139.717965
15   品川区旗の台  35.605336  139.702467
16   品川区東中延  35.609164  139.713319
17  品川区東五反田  35.627833  139.726544
18   品川区東八潮  35.621438  139.770490
19   品川区東品川  35.617538  139.747934
20   品川区東大井  35.603447  139.739232
21    品川区荏原  35.614979  139.706143
22   品川区西中延  35.610146  139.707784
23  品川区西五反田  35.624536  139.716994
24   品川区西品川  35.613134  139.727233
25   品川区西大井  35.598529  139.720886
26    品川区豊町  35.607751  139.720655


## 目黒区

In [195]:
df_meguro = pd.read_csv('../data/outer_data/13110_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_meguro)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/meguro_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0     目黒区三田  35.640047  139.712134
1    目黒区上目黒  35.642629  139.693567
2    目黒区下目黒  35.629252  139.706110
3    目黒区中央町  35.630742  139.689854
4     目黒区中根  35.615382  139.674388
5     目黒区中町  35.632433  139.695629
6    目黒区中目黒  35.638708  139.701286
7    目黒区五本木  35.635198  139.686996
8     目黒区八雲  35.619644  139.669097
9      目黒区南  35.614290  139.687251
10    目黒区原町  35.616249  139.694252
11   目黒区大岡山  35.610284  139.682591
12    目黒区大橋  35.653292  139.687150
13    目黒区平町  35.616341  139.680060
14   目黒区東が丘  35.629147  139.667310
15    目黒区東山  35.647698  139.689127
16  目黒区柿の木坂  35.623792  139.675779
17    目黒区洗足  35.613096  139.694108
18    目黒区目黒  35.634449  139.706329
19  目黒区目黒本町  35.622061  139.696849
20   目黒区碑文谷  35.621961  139.685229
21   目黒区祐天寺  35.637987  139.693560
22   目黒区緑が丘  35.608628  139.676631
23  目黒区自由が丘  35.610115  139.667554
24   目黒区青葉台  35.650199  139.693944
25    目黒区駒場  35.658239  139.684495
26    目黒区鷹番  35.628356  139.686090


## 大田区

In [196]:
df_ota = pd.read_csv('../data/outer_data/13111_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_ota)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/ota_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

             区町名       平均緯度        平均経度
0   大田区ふるさとの浜辺公園  35.572291  139.742604
1         大田区上池台  35.597313  139.697613
2         大田区下丸子  35.568172  139.685145
3          大田区中央  35.577475  139.716864
4         大田区中馬込  35.594857  139.706643
..           ...        ...         ...
57        大田区西糀谷  35.556609  139.733334
58        大田区西蒲田  35.566996  139.712205
59        大田区西馬込  35.588643  139.704690
60      大田区雪谷大塚町  35.592133  139.678965
61        大田区鵜の木  35.575544  139.679596

[62 rows x 3 columns]


## 世田谷区

In [197]:
df_setagaya = pd.read_csv('../data/outer_data/13112_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_setagaya)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/setagaya_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

         区町名       平均緯度        平均経度
0     世田谷区三宿  35.650962  139.674976
1   世田谷区三軒茶屋  35.641302  139.668089
2    世田谷区上北沢  35.668606  139.621786
3    世田谷区上用賀  35.634122  139.629218
4   世田谷区上祖師谷  35.658921  139.596879
..       ...        ...         ...
56    世田谷区野毛  35.603910  139.641424
57    世田谷区野沢  35.633201  139.670885
58    世田谷区鎌田  35.619734  139.613772
59    世田谷区駒沢  35.630763  139.656503
60  世田谷区駒沢公園  35.624331  139.660591

[61 rows x 3 columns]


## 渋谷区

In [198]:
df_shibuya = pd.read_csv('../data/outer_data/13113_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_shibuya)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/shibuya_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

          区町名       平均緯度        平均経度
0       渋谷区上原  35.666972  139.681107
1      渋谷区代々木  35.680578  139.694737
2   渋谷区代々木神園町  35.675626  139.695735
3     渋谷区代官山町  35.650831  139.704518
4    渋谷区元代々木町  35.671495  139.685587
5      渋谷区円山町  35.657421  139.694487
6       渋谷区初台  35.678170  139.685941
7     渋谷区千駄ケ谷  35.679486  139.707037
8     渋谷区南平台町  35.654160  139.696324
9      渋谷区大山町  35.669043  139.675067
10    渋谷区宇田川町  35.661766  139.697841
11     渋谷区富ケ谷  35.666346  139.688087
12     渋谷区幡ケ谷  35.678543  139.673774
13      渋谷区広尾  35.651587  139.717228
14     渋谷区恵比寿  35.645549  139.716720
15    渋谷区恵比寿南  35.644430  139.708556
16    渋谷区恵比寿西  35.648462  139.706475
17      渋谷区本町  35.684321  139.680383
18       渋谷区東  35.653476  139.709652
19      渋谷区松濤  35.660600  139.691832
20     渋谷区桜丘町  35.655504  139.701090
21      渋谷区渋谷  35.659050  139.705556
22     渋谷区猿楽町  35.650099  139.701810
23      渋谷区神南  35.664043  139.700020
24     渋谷区神宮前  35.669007  139.708117
25     渋谷区神山町  35.664133  139.691856
2

## 中野区

In [199]:
df_nakano = pd.read_csv('../data/outer_data/13114_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_nakano)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/nakano_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

       区町名       平均緯度        平均経度
0   中野区上高田  35.713379  139.674873
1   中野区上鷺宮  35.730543  139.631243
2    中野区中央  35.699807  139.674727
3    中野区中野  35.705471  139.669197
4    中野区丸山  35.724664  139.653563
5    中野区南台  35.682494  139.667425
6   中野区大和町  35.713091  139.648814
7   中野区弥生町  35.689305  139.673221
8    中野区新井  35.713745  139.665387
9    中野区本町  35.694774  139.673553
10  中野区東中野  35.706124  139.684346
11  中野区松が丘  35.720418  139.671167
12  中野区江原町  35.730163  139.672758
13  中野区江古田  35.725672  139.667031
14   中野区沼袋  35.720566  139.663482
15   中野区白鷺  35.719893  139.634838
16   中野区若宮  35.718862  139.645324
17   中野区野方  35.716769  139.654936
18   中野区鷺宮  35.724498  139.637845


## 杉並区

In [200]:
df_suginami = pd.read_csv('../data/outer_data/13115_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_suginami)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/suginami_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    杉並区上井草  35.721608  139.604907
1     杉並区上荻  35.707565  139.611184
2   杉並区上高井戸  35.674510  139.612215
3    杉並区下井草  35.720050  139.624514
4   杉並区下高井戸  35.671808  139.630893
5    杉並区久我山  35.687756  139.598631
6     杉並区井草  35.726785  139.617359
7     杉並区今川  35.716579  139.605835
8    杉並区南荻窪  35.699587  139.612946
9     杉並区和泉  35.677020  139.652127
10    杉並区和田  35.692930  139.660061
11   杉並区善福寺  35.714412  139.592315
12   杉並区堀ノ内  35.689203  139.651089
13    杉並区大宮  35.683776  139.642131
14    杉並区天沼  35.707981  139.624591
15    杉並区宮前  35.693115  139.607544
16   杉並区成田東  35.694137  139.636080
17   杉並区成田西  35.689829  139.629603
18    杉並区方南  35.679118  139.660302
19   杉並区本天沼  35.713908  139.625602
20   杉並区松ノ木  35.691629  139.643658
21    杉並区松庵  35.698685  139.595615
22    杉並区桃井  35.712617  139.609098
23    杉並区梅里  35.696633  139.646598
24    杉並区永福  35.674994  139.640851
25   杉並区浜田山  35.680925  139.629744
26    杉並区清水  35.714006  139.617617
27    杉並区荻窪  35.6978

## 豊島区

In [201]:
df_toshima = pd.read_csv('../data/outer_data/13116_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_toshima)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/toshima_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    豊島区上池袋  35.738207  139.719500
1    豊島区北大塚  35.733466  139.728304
2     豊島区千川  35.740408  139.692984
3     豊島区千早  35.734515  139.689062
4    豊島区南大塚  35.728767  139.731075
5    豊島区南池袋  35.725190  139.715276
6    豊島区南長崎  35.726933  139.685615
7     豊島区巣鴨  35.736094  139.735253
8    豊島区東池袋  35.729654  139.720546
9     豊島区池袋  35.735778  139.707245
10  豊島区池袋本町  35.742135  139.713073
11    豊島区目白  35.723234  139.703380
12   豊島区西巣鴨  35.740340  139.728063
13   豊島区西池袋  35.729156  139.704778
14    豊島区要町  35.736578  139.692653
15    豊島区長崎  35.731073  139.687135
16  豊島区雑司が谷  35.720299  139.718327
17    豊島区駒込  35.737936  139.744190
18    豊島区高松  35.738493  139.698601
19    豊島区高田  35.714976  139.712800


## 北区

In [202]:
df_kita = pd.read_csv('../data/outer_data/13117_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_kita)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/kita_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

       区町名       平均緯度        平均経度
0    北区上中里  35.746334  139.749023
1    北区上十条  35.762433  139.718770
2    北区中十条  35.762742  139.725339
3     北区中里  35.739160  139.751368
4   北区十条仲原  35.765568  139.720182
5    北区十条台  35.755916  139.724435
6     北区堀船  35.753739  139.747639
7    北区岩淵町  35.784545  139.723691
8     北区岸町  35.757852  139.732369
9     北区志茂  35.780050  139.730726
10   北区昭和町  35.747937  139.753901
11   北区東十条  35.766301  139.728198
12   北区東田端  35.739845  139.762035
13    北区栄町  35.750323  139.744048
14   北区桐ケ丘  35.779110  139.708704
15    北区浮間  35.788998  139.698850
16   北区滝野川  35.747388  139.728404
17    北区王子  35.760143  139.736879
18  北区王子本町  35.754302  139.730972
19    北区田端  35.736118  139.757144
20  北区田端新町  35.740459  139.765550
21    北区神谷  35.770641  139.731456
22   北区西が丘  35.768267  139.712799
23   北区西ケ原  35.743749  139.740347
24    北区豊島  35.762545  139.744181
25    北区赤羽  35.781687  139.722090
26   北区赤羽北  35.784076  139.708569
27   北区赤羽南  35.775282  139.723910
28   北区赤羽台  35

## 荒川区

In [203]:
df_arakawa = pd.read_csv('../data/outer_data/13118_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_arakawa)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/arakawa_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

       区町名       平均緯度        平均経度
0   荒川区南千住  35.734233  139.797009
1   荒川区東尾久  35.745773  139.770334
2  荒川区東日暮里  35.729465  139.780491
3    荒川区町屋  35.748008  139.781839
4    荒川区荒川  35.738257  139.781210
5   荒川区西尾久  35.749465  139.759099
6  荒川区西日暮里  35.733334  139.769966


## 板橋区

In [204]:
df_itabashi = pd.read_csv('../data/outer_data/13119_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_itabashi)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/itabashi_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

         区町名       平均緯度        平均経度
0      板橋区三園  35.791138  139.640998
1     板橋区上板橋  35.763673  139.674004
2     板橋区中丸町  35.741883  139.703456
3      板橋区中台  35.769608  139.679761
4     板橋区中板橋  35.755978  139.697483
5      板橋区仲宿  35.755496  139.710608
6      板橋区仲町  35.752715  139.697417
7     板橋区前野町  35.768375  139.689460
8      板橋区加賀  35.756145  139.716667
9    板橋区南常盤台  35.757458  139.687627
10     板橋区南町  35.739965  139.701406
11    板橋区双葉町  35.758290  139.700067
12     板橋区向原  35.741888  139.683185
13     板橋区四葉  35.779643  139.652305
14     板橋区坂下  35.783406  139.683763
15    板橋区大原町  35.768862  139.698342
16    板橋区大和町  35.758855  139.704645
17   板橋区大山東町  35.749111  139.705745
18    板橋区大山町  35.748849  139.699572
19   板橋区大山西町  35.746932  139.695226
20  板橋区大山金井町  35.746020  139.705516
21    板橋区大谷口  35.744528  139.689014
22  板橋区大谷口上町  35.748433  139.691382
23  板橋区大谷口北町  35.750546  139.688498
24     板橋区大門  35.783842  139.648351
25    板橋区宮本町  35.763852  139.701548
26   板橋区富士見町  35.760947  139

## 練馬区

In [205]:
df_nerima = pd.read_csv('../data/outer_data/13120_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_nerima)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/nerima_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

          区町名       平均緯度        平均経度
0      練馬区三原台  35.753678  139.603666
1     練馬区上石神井  35.728273  139.592441
2   練馬区上石神井南町  35.723215  139.595672
3     練馬区下石神井  35.732153  139.605551
4       練馬区中村  35.732547  139.642960
5      練馬区中村北  35.735565  139.641895
6      練馬区中村南  35.728484  139.644223
7      練馬区光が丘  35.760006  139.628941
8       練馬区北町  35.764696  139.656203
9      練馬区南大泉  35.744517  139.573305
10     練馬区南田中  35.735338  139.615390
11      練馬区向山  35.741610  139.640834
12     練馬区土支田  35.763151  139.613103
13   練馬区大泉学園町  35.766119  139.587186
14     練馬区大泉町  35.763558  139.600484
15    練馬区富士見台  35.739057  139.622663
16     練馬区小竹町  35.741553  139.675928
17     練馬区平和台  35.758664  139.661377
18      練馬区早宮  35.751072  139.654000
19      練馬区旭丘  35.735042  139.676461
20      練馬区旭町  35.769761  139.625594
21     練馬区春日町  35.752024  139.641786
22     練馬区東大泉  35.748963  139.587161
23      練馬区栄町  35.738727  139.670074
24      練馬区桜台  35.743665  139.662468
25     練馬区氷川台  35.753627  139.667154
2

## 足立区

In [206]:
df_adachi = pd.read_csv('../data/outer_data/13121_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_adachi)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/adachi_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0    足立区一ツ家  35.781572  139.812957
1   足立区中央本町  35.773638  139.806966
2     足立区中川  35.770379  139.852018
3     足立区伊興  35.794732  139.783771
4   足立区伊興本町  35.799345  139.787836
..      ...        ...         ...
81    足立区足立  35.763144  139.808146
82    足立区辰沼  35.787673  139.834868
83    足立区関原  35.768113  139.788812
84    足立区青井  35.774205  139.816978
85    足立区鹿浜  35.783508  139.755509

[86 rows x 3 columns]


## 葛飾区

In [207]:
df_katsushika = pd.read_csv('../data/outer_data/13122_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_katsushika)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/katsushika_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

        区町名       平均緯度        平均経度
0   葛飾区お花茶屋  35.751508  139.840511
1     葛飾区亀有  35.763256  139.848080
2    葛飾区南水元  35.776963  139.860156
3    葛飾区四つ木  35.736901  139.835584
4     葛飾区堀切  35.747838  139.828787
5     葛飾区奥戸  35.732895  139.863493
6     葛飾区宝町  35.744824  139.836978
7     葛飾区小菅  35.754835  139.822712
8     葛飾区新宿  35.762144  139.861023
9    葛飾区新小岩  35.716545  139.862576
10  葛飾区東四つ木  35.730608  139.840310
11   葛飾区東堀切  35.752379  139.835913
12  葛飾区東新小岩  35.724630  139.858929
13   葛飾区東水元  35.785048  139.867223
14   葛飾区東立石  35.733916  139.848512
15   葛飾区東金町  35.775089  139.876124
16    葛飾区柴又  35.755673  139.875838
17    葛飾区水元  35.785214  139.859964
18  葛飾区水元公園  35.789509  139.870365
19    葛飾区白鳥  35.750610  139.845169
20    葛飾区立石  35.740224  139.848150
21    葛飾区細田  35.741211  139.871545
22   葛飾区西亀有  35.761346  139.837560
23  葛飾区西新小岩  35.721685  139.849996
24   葛飾区西水元  35.787625  139.851069
25    葛飾区金町  35.765961  139.870626
26    葛飾区鎌倉  35.745921  139.878776
27    葛飾区青戸  35.7497

## 江戸川区

In [208]:
df_edogawa = pd.read_csv('../data/outer_data/13123_2021.csv', encoding='shift-jis')
ward_infos = get_ward_infos(df_edogawa)
mean_latitude_longitudes = get_mean_latitude_longitude(ward_infos)
mean_latitude_longitudes.to_csv('../data/handmade_data/edogawa_latitude_longitude.csv', index=False)
print(mean_latitude_longitudes)

          区町名       平均緯度        平均経度
0     江戸川区一之江  35.690134  139.880767
1    江戸川区一之江町  35.683300  139.873161
2     江戸川区上一色  35.726509  139.872380
3     江戸川区上篠崎  35.711183  139.901325
4    江戸川区下篠崎町  35.698643  139.913287
5      江戸川区中央  35.708920  139.871437
6     江戸川区中葛西  35.664471  139.869424
7    江戸川区二之江町  35.681825  139.873068
8     江戸川区北小岩  35.741929  139.889073
9     江戸川区北篠崎  35.718534  139.896082
10    江戸川区北葛西  35.675196  139.860524
11    江戸川区南小岩  35.726180  139.884027
12   江戸川区南篠崎町  35.694162  139.903685
13    江戸川区南葛西  35.649709  139.874132
14     江戸川区大杉  35.707605  139.879326
15   江戸川区宇喜田町  35.676969  139.863165
16    江戸川区小松川  35.698100  139.850950
17     江戸川区平井  35.707345  139.843738
18     江戸川区新堀  35.704484  139.889050
19    江戸川区春江町  35.689760  139.883723
20    江戸川区本一色  35.718602  139.871303
21    江戸川区東小岩  35.729258  139.890845
22   江戸川区東小松川  35.693396  139.864787
23    江戸川区東松本  35.718805  139.884027
24    江戸川区東瑞江  35.688309  139.898119
25    江戸川区東篠崎  35.696517  139.914754
2